# awsmt
> Library to use Amazon Web Service machine translation services

In [ ]:
#| default_exp awsmt

In [ ]:
#| hide
import os

In [ ]:
#| hide
running_in_colab = 'google.colab' in str(get_ipython())
if running_in_colab:
    from google.colab import drive
    drive.mount('/content/drive')
    homedir = "/content/drive/MyDrive"
else:
    homedir = os.getenv('HOME')

In [ ]:
#| hide
if running_in_colab:
    github_test_folder = homedir+"/github/polyglottech/mteval"
    %cd {github_test_folder}
    !pip3 install nbdev
    !pip3 install -e '.[dev]'

In [ ]:
#| hide
from dotenv import load_dotenv

In [ ]:
#| hide
if running_in_colab:
    # Colab doesn't have a mechanism to set environment variables other than python-dotenv
    env_file = homedir+'/secrets/.env'
    %load_ext dotenv
    %dotenv {env_file}

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import os
import boto3
import time
from pathlib import Path
from langcodes import *

class awstranslate:
    """Class to get translations from the Amazon Web Service Translate API"""
    def __init__(self):
        """Constructor of awstranslate class"""
        # Authentication via environment variables
        self._translate_client = boto3.client(service_name='translate', use_ssl=True)
        
        # Initializing the list of supported languages (source or target)
        self._supported_langs = [x["LanguageCode"] for x in self._translate_client.list_languages()["Languages"]]

    def check_langpair(self, sourcelang, targetlang):
        """Function to verify if a language pair identified by language ids is supported"""
        supported = False
        if tag_is_valid(sourcelang) and tag_is_valid(targetlang):
            if sourcelang in self._supported_langs and targetlang in self._supported_langs:
                supported = True
            
        return supported
    
    def translate_text(self,sourcelang, targetlang, text):
        """Function to translate text into the target language"""
        
        translated_text = ""
        if self.check_langpair(sourcelang,targetlang):
            result = self._translate_client.translate_text(Text=text, SourceLanguageCode=sourcelang, TargetLanguageCode=targetlang)
            translated_text = result.get('TranslatedText')
        
        return translated_text
       


In [ ]:
#| hide
awsmt = awstranslate()
supported = awsmt.check_langpair("en","it")
assert supported == True 
translation = awsmt.translate_text("en","it","This is a cool car!")
assert translation == "Questa è una bella macchina!"

In [ ]:
show_doc(awstranslate.check_langpair)
show_doc(awstranslate.translate_text)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()